# About
This notebook shows how to utilize the custom sklearn transformer to 
- derive regular season wins & losses per team per season
- derive ncaa historic wins & losses per team 
- calculate winning ods based on different win / loss compositions
- derive seed per team per season

In [1]:
import os
os.chdir("/home/jovyan/work")

import pandas as pd

from src.utils import get_table
from src.transformer import HistWinLossTransformer, OddTransformer, SeedTransformer, WinLossTransformer

***
# Regular Season

In [2]:
df_test = pd.DataFrame([[2014, 1444, 1450], [2014, 1450, 1444]], columns=["season", "team_id", "opponent_id"])
wlt = WinLossTransformer(team_id_a="team_id", team_id_b="opponent_id", split_by_ot=False)
wlt.fit_transform(df_test)

,season,team_id,opponent_id,wins_reg_a,wins_reg_b,losses_reg_a,losses_reg_b
0,2014,1444,1450,22,9,9,21
1,2014,1450,1444,9,22,21,9


***
# Historic NCAA

In [3]:
df_test = pd.DataFrame([[2014, 1444, 1450], [2014, 1450, 1444]], columns=["season", "team_id", "opponent_id"])
hwlt = HistWinLossTransformer(team_id_a="team_id", team_id_b="opponent_id", split_by_ot=True)
hwlt.fit_transform(df_test)

,season,team_id,opponent_id,wins_no_ot_hist_a,wins_ot_hist_a,wins_no_ot_hist_b,wins_ot_hist_b,losses_no_ot_hist_a,losses_ot_hist_a,losses_no_ot_hist_b,losses_ot_hist_b
0,2014,1444,1450,1,0,3,0,3,0,2,1
1,2014,1450,1444,3,0,1,0,2,1,3,0


***
# Derive winning odds

In [4]:
df_transformed = wlt.fit_transform(df_test)
ot = OddTransformer(
    ["wins_reg_a"],
    ["losses_reg_a"],
    ["wins_reg_b"],
    ["losses_reg_b"]
)
df_odds = ot.fit_transform(df_transformed)
df_odds

,season,team_id,opponent_id,wins_reg_a,wins_reg_b,losses_reg_a,losses_reg_b,odds_a
0,2014,1444,1450,22,9,9,21,0.9993
1,2014,1450,1444,9,22,21,9,0.0005


***
## Seeding

In [5]:
st = SeedTransformer(team_id_a="team_id", team_id_b="opponent_id")
st.fit_transform(df_test)

,season,team_id,opponent_id,seed_rank_a,seed_region_a,seed_rank_b,seed_region_b
0,2014,1444,1450,14.0,X,99.0,NOSEED
1,2014,1450,1444,99.0,NOSEED,14.0,X
